<a href="https://colab.research.google.com/github/AnumandlaS/Soccer-Player-Reidentification/blob/main/Re_identification(in_a_single_feed)using_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import pytesseract
from collections import defaultdict

# Load YOLOv8 model
model = YOLO('/content/best.pt')

# Initialize ID mapping
number_to_id = {}  # Maps detected number (e.g., "23") to global ID
global_id_counter = 0  # Increments for new IDs
track_id_to_global_id = {}  # Maps YOLO track ID to global ID for fallback

# Load video
video_path = '/content/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer
out = cv2.VideoWriter('output_reid_ocr.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

def preprocess_ocr_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to enhance text
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extract_number(img):
    # Use OCR to detect text (numbers) in the image
    config = '--psm 6 digits'  # Assume single line of digits
    text = pytesseract.image_to_string(img, config=config).strip()
    # Extract only digits to avoid noise
    digits = ''.join(filter(str.isdigit, text))
    return digits if digits else None

ret = True
frame_num = 0
while ret:
    ret, frame = cap.read()
    if ret:
        # Detect and track people
        results = model.track(frame, persist=True)
        boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes
        track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

        # Process each detected person
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            track_id = int(track_ids[i]) if i < len(track_ids) else -1

            # Crop the lower half of the bounding box (likely where shorts are)
            shorts_region = frame[y1 + int((y2 - y1) * 0.5):y2, x1:x2]
            if shorts_region.size == 0:
                continue

            # Preprocess and apply OCR
            shorts_processed = preprocess_ocr_image(shorts_region)
            number = extract_number(shorts_processed)

            # Assign global ID based on OCR or fallback to track ID
            if number and number in number_to_id:
                global_id = number_to_id[number]
            elif number:
                global_id_counter += 1
                number_to_id[number] = global_id_counter
                global_id = global_id_counter
            else:
                # Fallback to track ID if OCR fails
                if track_id != -1 and track_id in track_id_to_global_id:
                    global_id = track_id_to_global_id[track_id]
                else:
                    global_id_counter += 1
                    global_id = global_id_counter
                    if track_id != -1:
                        track_id_to_global_id[track_id] = global_id

            # Draw bounding box and global ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {global_id}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Write frame to output video
        out.write(frame)
        frame_num += 1

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 1 ball, 16 players, 2 referees, 110.3ms
Speed: 26.5ms preprocess, 110.3ms inference, 419.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 players, 2 referees, 67.5ms
Speed: 2.8ms preprocess, 67.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 67.5ms
Speed: 3.1ms preprocess, 67.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 2 referees, 67.